First step is to import the relevant pandas package and load the wiki(HTML) data.
To obtain the proper columns only we have to slice tables as tables[0]

In [3]:
import pandas as pd
tables=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df=pd.DataFrame(tables[0])

In [4]:
print(df)

    Postcode           Borough          Neighbourhood
0        M1A      Not assigned           Not assigned
1        M2A      Not assigned           Not assigned
2        M3A        North York              Parkwoods
3        M4A        North York       Victoria Village
4        M5A  Downtown Toronto           Harbourfront
..       ...               ...                    ...
283      M8Z         Etobicoke              Mimico NW
284      M8Z         Etobicoke     The Queensway West
285      M8Z         Etobicoke  Royal York South West
286      M8Z         Etobicoke         South of Bloor
287      M9Z      Not assigned           Not assigned

[288 rows x 3 columns]


Next we have to select only the rows in the dataframe which have a value in the "Borough' column which is unequal to "Not assigned". The remaining rows are included in dataframe df2

In [5]:
df2=df[df['Borough'] != "Not assigned"]
print(df2)

    Postcode           Borough             Neighbourhood
2        M3A        North York                 Parkwoods
3        M4A        North York          Victoria Village
4        M5A  Downtown Toronto              Harbourfront
5        M5A  Downtown Toronto               Regent Park
6        M6A        North York          Lawrence Heights
..       ...               ...                       ...
282      M8Z         Etobicoke  Kingsway Park South West
283      M8Z         Etobicoke                 Mimico NW
284      M8Z         Etobicoke        The Queensway West
285      M8Z         Etobicoke     Royal York South West
286      M8Z         Etobicoke            South of Bloor

[211 rows x 3 columns]


After this we have to eliminate the double items in (combined) 'Postcode' and 'Borough' columns. The neighbourhood items are combined and seperated by a ','.   

In [6]:
df3=df2.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()

Next we have identify the 'Not assigned' rows in the "Neighbourhood' column. These need to be replaced with the value in the 'Borough' column. As seen below this involves only 1 item, which is replaced by the related 'Borough' field contents.

In [7]:
print(df3[df3['Neighbourhood']=='Not assigned'])

   Postcode       Borough Neighbourhood
85      M7A  Queen's Park  Not assigned


In [8]:
df3['Neighbourhood']=df3['Neighbourhood'].str.replace("Not assigned","Queen's Park")

In [9]:
print(df3)

    Postcode      Borough                                      Neighbourhood
0        M1B  Scarborough                                      Rouge,Malvern
1        M1C  Scarborough               Highland Creek,Rouge Hill,Port Union
2        M1E  Scarborough                    Guildwood,Morningside,West Hill
3        M1G  Scarborough                                             Woburn
4        M1H  Scarborough                                          Cedarbrae
..       ...          ...                                                ...
98       M9N         York                                             Weston
99       M9P    Etobicoke                                          Westmount
100      M9R    Etobicoke  Kingsview Village,Martin Grove Gardens,Richvie...
101      M9V    Etobicoke  Albion Gardens,Beaumond Heights,Humbergate,Jam...
102      M9W    Etobicoke                                          Northwest

[103 rows x 3 columns]


Using the shape method we see that there are 103 rows remaining in the dataset after all transformations have been performed

In [10]:
df3.shape

(103, 3)

The next part of the assignment is to load the geospatial dataset and merge with the wiki dataset(df3).
First step is to load the csv and transform to dataframe.

In [11]:
geo=pd.read_csv('Geospatial_Coordinates.csv')

In [12]:
df_geo=pd.DataFrame(geo)
print(df_geo)

    Postal Code   Latitude  Longitude
0           M1B  43.806686 -79.194353
1           M1C  43.784535 -79.160497
2           M1E  43.763573 -79.188711
3           M1G  43.770992 -79.216917
4           M1H  43.773136 -79.239476
..          ...        ...        ...
98          M9N  43.706876 -79.518188
99          M9P  43.696319 -79.532242
100         M9R  43.688905 -79.554724
101         M9V  43.739416 -79.588437
102         M9W  43.706748 -79.594054

[103 rows x 3 columns]


In [ ]:
The 2 dataframes(df3 and df_geo) are merged using the merge function in pandas using the Postcode/Postal Code keys. As default
we apply an inner join (default setting) 

In [14]:
df_comb=df3.merge(df_geo, left_on='Postcode', right_on='Postal Code')

After the merging of the 2 dataframes we check that we have the same number of rows left from df3 which is indeed the case (103 rows before and after merging). We drop the 'Postal Code' column as the values are the same as the 'Postcode' column.

In [22]:
df_comb=df_comb.drop(columns=['Postal Code'])

In [23]:
print(df_comb)

    Postcode      Borough                                      Neighbourhood  \
0        M1B  Scarborough                                      Rouge,Malvern   
1        M1C  Scarborough               Highland Creek,Rouge Hill,Port Union   
2        M1E  Scarborough                    Guildwood,Morningside,West Hill   
3        M1G  Scarborough                                             Woburn   
4        M1H  Scarborough                                          Cedarbrae   
..       ...          ...                                                ...   
98       M9N         York                                             Weston   
99       M9P    Etobicoke                                          Westmount   
100      M9R    Etobicoke  Kingsview Village,Martin Grove Gardens,Richvie...   
101      M9V    Etobicoke  Albion Gardens,Beaumond Heights,Humbergate,Jam...   
102      M9W    Etobicoke                                          Northwest   

      Latitude  Longitude  
0    43.806